In [1]:
import numpy as np
import pandas as pd
import os
import shutil
import random
import torch
import torchvision
import matplotlib.pyplot as plt
import time
import copy

from torchvision import transforms, models
from tqdm import tqdm_notebook as tqdm

data_root = './data/initial'
data_generated = './data/generated'
train_dir = './data/train'
val_dir = './data/val'
test_dir = './data/initial/test'
model_dir = './best_model'

random.seed(0)
np.random.seed(0)
torch.manual_seed(0)
torch.cuda.manual_seed(0)
torch.backends.cudnn.deterministic = True

In [2]:
train_transforms = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

val_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

train_dataset = torchvision.datasets.ImageFolder(os.path.join(data_generated, 'train'), train_transforms)
val_dataset = torchvision.datasets.ImageFolder(val_dir, val_transforms)

In [3]:
def validate_model(model, loss, dataloader, device):
    was_training = model.training
    model.eval()
    running_loss = 0
    running_acc = 0
    for inputs, labels in dataloader:
        inputs = inputs.to(device)
        labels = labels.to(device)
        with torch.set_grad_enabled(False):
            preds = model(inputs)
            loss_value = loss(preds, labels)
            preds_class = preds.argmax(dim=1)
            running_loss += loss_value.item()
            running_acc += (preds_class == labels.data).float().mean()
    epoch_loss = running_loss / len(dataloader)
    epoch_acc = running_acc / len(dataloader)
    if was_training:
        model.train()
    return epoch_loss, epoch_acc

def save_model(model, val_acc, path):
    shutil.rmtree(path, ignore_errors=True)
    os.makedirs(path)
    model_name = 'model_{:.3f}.torch'.format(val_acc)
    torch.save({'state_dict': model.state_dict()}, os.path.join(path, model_name))

def load_model(model_cls, path, device):
    model = model_cls()
    checkpoint = torch.load(path)
    model.load_state_dict(checkpoint['state_dict'])
    model.eval()
    return model.to(device)

In [4]:
def train_model(model, loss, optimizer, scheduler, num_epochs, eval_every_nth_batch):
    best_val_acc = 0
    for epoch in range(num_epochs):
        print('Epoch {}/{}:'.format(epoch, num_epochs - 1), flush=True)

        dataloader = train_dataloader
        model.train()

        running_loss = 0
        running_acc = 0
        batch_id = 0
        n_samples_passed = 0
        for inputs, labels in tqdm(dataloader):
            scheduler.step()
            inputs = inputs.to(device)
            labels = labels.to(device)
            optimizer.zero_grad()

            preds = model(inputs)
            loss_value = loss(preds, labels)
            preds_class = preds.argmax(dim=1)

            loss_value.backward()
            optimizer.step()

            running_loss += loss_value.item()
            running_acc += (preds_class == labels.data).float().mean()
            batch_id += 1
            if batch_id % eval_every_nth_batch == 0:
                train_loss = running_loss / eval_every_nth_batch
                train_acc = running_acc / eval_every_nth_batch
                
                val_loss, val_acc = validate_model(model, loss, val_dataloader, device)
                is_best = val_acc > best_val_acc
                if is_best:
                    save_model(model, val_acc, model_dir)
                    best_val_acc = val_acc
                    val_loss_msg = 'Valid loss: {:.4f} Valid acc: {:.4f} ***'.format(val_loss, val_acc)
                    val_loss_msg += ' CURRENTLY THE BEST!'
                    print('Train loss: {:.4f} Train acc: {:.4f}'.format(train_loss, train_acc), flush=True)
                    print(val_loss_msg, flush=True)
                
                running_loss = 0
                running_acc = 0

    return model

In [79]:
models.resnet18(pretrained=True)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace)
      (conv2): Co

In [20]:
class Model(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.__head = models.resnet50(pretrained=True)
        for param in self.__head.parameters():
            param.requires_grad = False
        #for param in self.__head.layer4[1].parameters():
        #    param.requires_grad = True
        #self.__head.fc = torch.nn.Linear(model.fc.in_features, 2)
        
        self.__bottom = torch.nn.Sequential()
        #self.__bottom.add_module("linear_1_dropout", torch.nn.Dropout(p=0.1))
        #self.__bottom.add_module("linear_1", torch.nn.Linear(self.__head.fc.in_features, 2))
        self.__bottom.add_module("linear_1", torch.nn.Linear(self.__head.fc.in_features, 256))
        self.__bottom.add_module("linear_1_dropout", torch.nn.Dropout(p=0.5))
        self.__bottom.add_module("relu_1", torch.nn.ReLU(inplace=True))
        self.__bottom.add_module("linear_2", torch.nn.Linear(256, 2))
        
        self.__head.fc = self.__bottom
    
    def forward(self, x):
        return self.__head.forward(x)

model = Model()

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

loss = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3, amsgrad=True, weight_decay=1e-5)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=100500, gamma=0.1)

In [21]:
batch_size = 64
eval_every_nth_batch = 1
n_workers = 4
train_dataloader = torch.utils.data.DataLoader(
    train_dataset, batch_size=batch_size, shuffle=True, num_workers=n_workers)
val_dataloader = torch.utils.data.DataLoader(
    val_dataset, batch_size=batch_size, shuffle=False, num_workers=n_workers)

train_model(model, loss, optimizer, scheduler, num_epochs=1, eval_every_nth_batch=eval_every_nth_batch);

Epoch 0/0:


A Jupyter Widget

Train loss: 0.7096 Train acc: 0.4375
Valid loss: 1.2211 Valid acc: 0.4219 *** CURRENTLY THE BEST!
Train loss: 1.0162 Train acc: 0.5469
Valid loss: 0.6393 Valid acc: 0.5781 *** CURRENTLY THE BEST!
Train loss: 0.8144 Train acc: 0.4219
Valid loss: 0.6096 Valid acc: 0.5820 *** CURRENTLY THE BEST!
Train loss: 0.5756 Train acc: 0.6562
Valid loss: 0.6112 Valid acc: 0.7044 *** CURRENTLY THE BEST!
Train loss: 0.5877 Train acc: 0.7344
Valid loss: 0.5839 Valid acc: 0.8008 *** CURRENTLY THE BEST!
Train loss: 0.5646 Train acc: 0.7031
Valid loss: 0.5572 Valid acc: 0.8138 *** CURRENTLY THE BEST!
Train loss: 0.5145 Train acc: 0.7188
Valid loss: 0.5351 Valid acc: 0.8190 *** CURRENTLY THE BEST!
Train loss: 0.0740 Train acc: 0.9844
Valid loss: 0.3793 Valid acc: 0.8411 *** CURRENTLY THE BEST!
Train loss: 0.0124 Train acc: 1.0000
Valid loss: 0.4042 Valid acc: 0.8464 *** CURRENTLY THE BEST!
Train loss: 0.0625 Train acc: 0.9844
Valid loss: 0.3921 Valid acc: 0.8568 *** CURRENTLY THE BEST!
Train loss: 0.0208 T

In [12]:
model = load_model(Model, './best_model/model_0.868.torch', device)

In [14]:
class ImageFolderWithPaths(torchvision.datasets.ImageFolder):
    def __getitem__(self, index):
        original_tuple = super(ImageFolderWithPaths, self).__getitem__(index)
        path = os.path.basename(self.imgs[index][0])
        path = path.split()[0]
        tuple_with_path = (original_tuple + (path,))
        return tuple_with_path
    
test_dataset = ImageFolderWithPaths(test_dir, val_transforms)
test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size=64, shuffle=False, num_workers=0)

In [15]:
model.eval()

test_predictions = []
test_img_paths = []
for inputs, labels, paths in tqdm(test_dataloader):
    inputs = inputs.to(device)
    labels = labels.to(device)
    with torch.set_grad_enabled(False):
        preds = model(inputs)
    test_predictions.append(
        torch.nn.functional.softmax(preds, dim=1)[:,1].data.cpu().numpy())
    test_img_paths.extend(paths)
    
test_predictions = np.concatenate(test_predictions)

A Jupyter Widget

In [16]:
submission_df = pd.DataFrame.from_dict({'id': test_img_paths, 'label': test_predictions})

In [17]:
submission_df['label'] = submission_df['label'].map(lambda pred: 'dirty' if pred > 0.5 else 'cleaned')
submission_df['id'] = submission_df['id'].str.replace('test/unknown/', '')
submission_df['id'] = submission_df['id'].str.replace('.jpg', '')
submission_df.set_index('id', inplace=True)
submission_df.head(n=6)

,label
id,
0000,dirty
0001,dirty
0002,dirty
0003,dirty
0004,dirty
0005,dirty


In [18]:
submission_df.to_csv('submission.csv')

# Data Preparation

In [12]:
class_names = ['cleaned', 'dirty']

for dir_name in [train_dir, val_dir]:
    for class_name in class_names:
        os.makedirs(os.path.join(dir_name, class_name), exist_ok=True)

for class_name in class_names:
    source_dir = os.path.join(data_root, 'train', class_name)
    for i, file_name in enumerate(tqdm(os.listdir(source_dir))):
        if i % 6 != 0:
            dest_dir = os.path.join(train_dir, class_name) 
        else:
            dest_dir = os.path.join(val_dir, class_name)
        shutil.copy(os.path.join(source_dir, file_name), os.path.join(dest_dir, file_name))

100%|███████████████████████████████████████| 20/20 [00:00<00:00, 172.42it/s]


Let's generate some samples

In [3]:
random.seed(0)
np.random.seed(0)
torch.manual_seed(0)
torch.cuda.manual_seed(0)
torch.backends.cudnn.deterministic = True

def prepare_dirs(root_path):
    shutil.rmtree(root_path, ignore_errors=True)
    os.makedirs(os.path.join(root_path, 'dirty'))
    os.makedirs(os.path.join(root_path, 'clean'))

def make_dataset(directory):
    ts = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ColorJitter(brightness=0.3, hue=0.25),
        transforms.RandomHorizontalFlip(),
        transforms.RandomVerticalFlip(),
        transforms.RandomRotation(180.0),
        transforms.RandomAffine(degrees=0, translate=(0.15, 0.15))
    ])
    return torchvision.datasets.ImageFolder(directory, ts)

def make_train_dataset():
    return make_dataset(train_dir)

def make_val_dataset():
    return make_dataset(val_dir)

def generate_images(n_amount, dataset_factory, dataset_type, root_path):
    path = os.path.join(root_path, dataset_type)
    prepare_dirs(path)
    name_pattern = '{{:0{}d}}.png'.format(len(str(n_amount)))
    class_path = {0: 'clean', 1: 'dirty'}
    image_id = 0
    while True:
        dataset = dataset_factory()
        data_iter = iter(dataset)
        for image, class_id in data_iter:
            if image_id >= n_amount:
                return
            image_name = name_pattern.format(image_id)
            image, class_id = next(data_iter)
            image.save(os.path.join(path, class_path[class_id], image_name), 'PNG')
            image_id += 1

In [4]:
n_gen_images_train = 262144
n_gen_images_val = 4096

generate_images(n_gen_images_train, make_train_dataset, 'train', data_generated)
#generate_images(n_gen_images_val, make_val_dataset, 'val', data_generated)